# Cat Vs Dog using VGG16

In [133]:
%matplotlib inline

import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Flatten, Dense
from keras.callbacks import Callback, ModelCheckpoint
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

import warnings
warnings.filterwarnings('ignore')

# Loading Pretrained Model

In [134]:
model_vgg16_conv = VGG16(weights='imagenet', include_top=False)

In [135]:
for layer in model_vgg16_conv.layers:
    layer.trainable = False

# Dataset Loading and Parameters

In [136]:
img_width, img_height = 150, 150
train_data_dir = 'cats-and-dogs/train'
val_data_dir = 'cats-and-dogs/test'
model_weights_file = 'vgg16-xfer-weights.h5'
nb_train_samples = 4
nb_val_samples = 4
nb_epochs = 5

# Model Building

In [137]:
input_layer = Input(shape=(img_width, img_height, 3))
output_vgg16_conv = model_vgg16_conv(input_layer)
x = Flatten()(output_vgg16_conv)
x = Dense(64, activation='relu')(x)
x = Dense(2, activation='softmax')(x)
model = Model(inputs=input_layer, outputs=x)

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [138]:
model.summary()

Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_34 (InputLayer)       [(None, 150, 150, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, None, None, 512)   14714688  
                                                                 
 flatten_16 (Flatten)        (None, 8192)              0         
                                                                 
 dense_32 (Dense)            (None, 64)                524352    
                                                                 
 dense_33 (Dense)            (None, 2)                 130       
                                                                 
Total params: 15239170 (58.13 MB)
Trainable params: 524482 (2.00 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


# Data Augmentation and Generators

In [139]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_width, img_height), 
                                                    batch_size=4, class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(val_data_dir, target_size=(img_width, img_height), 
                                                        batch_size=4,class_mode='categorical')

Found 1004 images belonging to 2 classes.
Found 10 images belonging to 2 classes.


# Model Training

In [140]:
callbacks = [ModelCheckpoint(model_weights_file, monitor='val_acc', save_best_only=True)]

history = model.fit_generator( train_generator, callbacks = callbacks, steps_per_epoch=nb_train_samples, 
                              epochs=40, validation_data=validation_generator, validation_steps=nb_val_samples)

print('Training Completed!')

Epoch 1/40
4/4 [==============================] - 2s 281ms/step - loss: 3.0150 - accuracy: 0.5000 - val_loss: 0.7341 - val_accuracy: 0.6000
Epoch 2/40
4/4 [==============================] - 1s 120ms/step - loss: 0.8351 - accuracy: 0.6250
Epoch 3/40
4/4 [==============================] - 1s 117ms/step - loss: 0.9468 - accuracy: 0.5625
Epoch 4/40
4/4 [==============================] - 0s 119ms/step - loss: 0.4397 - accuracy: 0.8750
Epoch 5/40
4/4 [==============================] - 0s 116ms/step - loss: 0.7487 - accuracy: 0.7500
Epoch 6/40
4/4 [==============================] - 0s 119ms/step - loss: 0.5244 - accuracy: 0.7500
Epoch 7/40
4/4 [==============================] - 1s 127ms/step - loss: 0.4960 - accuracy: 0.7500
Epoch 8/40
4/4 [==============================] - 1s 119ms/step - loss: 0.2536 - accuracy: 0.9375
Epoch 9/40
4/4 [==============================] - 0s 117ms/step - loss: 0.9659 - accuracy: 0.5625
Epoch 10/40
4/4 [==============================] - 1s 123ms/step - loss: 0.4

In [141]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
from os import listdir
from os.path import isfile, join

# Prediction and Evaluation

In [142]:
mypath = "cats-and-dogs/valid/"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
label = ['cat','dog']
print(onlyfiles)
count=0
total=len(onlyfiles)
# predicting images
dog_counter = 0 
cat_counter  = 0
for file in onlyfiles:
    img = image.load_img(mypath+file, target_size=(150,150))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
   
    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    l=classes[0]
    print("actual: "+file[:3]+" | prdicted: "+label[list(l).index(max(l))])
    if(file[:3]==label[list(l).index(max(l))]):
        count+=1
print(count/total)
    


['cat.57.jpg', 'cat.58.jpg', 'cat.60.jpg', 'cat.70.jpg', 'cat.76.jpg', 'cat.78.jpg', 'cat.93.jpg', 'cat.94.jpg', 'dog.492.jpg', 'dog.493.jpg', 'dog.494.jpg', 'dog.496.jpg', 'dog.514.jpg', 'dog.531.jpg', 'dog.614.jpg', 'dog.618.jpg']


1/1 [==============================] - 0s 179ms/step
actual: cat | prdicted: cat
1/1 [==============================] - 0s 52ms/step
actual: cat | prdicted: dog
1/1 [==============================] - 0s 49ms/step
actual: cat | prdicted: cat
1/1 [==============================] - 0s 52ms/step
actual: cat | prdicted: dog
1/1 [==============================] - 0s 48ms/step
actual: cat | prdicted: dog
1/1 [==============================] - 0s 50ms/step
actual: cat | prdicted: dog
1/1 [==============================] - 0s 49ms/step
actual: cat | prdicted: cat
1/1 [==============================] - 0s 48ms/step
actual: cat | prdicted: cat
1/1 [==============================] - 0s 50ms/step
actual: dog | prdicted: dog
1/1 [==============================] - 0s 50ms/step
actual: dog | prdicted: dog
1/1 [==============================] - 0s 50ms/step
actual: dog | prdicted: dog
1/1 [==============================] - 0s 49ms/step
actual: dog | prdicted: dog
1/1 [==============================] - 

# Finished